In [2]:
!pip install mmcv dotmap

  Using cached mmcv-1.1.4.tar.gz (239 kB)
  Using cached addict-2.3.0-py3-none-any.whl (3.7 kB)
Processing /home/hongky/.cache/pip/wheels/e5/9d/ad/2ee53cf262cba1ffd8afe1487eef788ea3f260b7e6232a80fc/PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl
  Using cached yapf-0.30.0-py2.py3-none-any.whl (190 kB)
  Created wheel for mmcv: filename=mmcv-1.1.4-cp36-cp36m-linux_x86_64.whl size=401669 sha256=341883e4f2ebaccc5fcb9aa7d39d8ee1104bc578df5a451c4022ed92ba9e19f0
  Stored in directory: /home/hongky/.cache/pip/wheels/25/ed/b5/2c2be1d1a3b05855c5b9c0507b95fff40326e1a84ebc1fa6b9
Successfully built mmcv


In [1]:
from __future__ import print_function
import argparse
import numpy as np
import os
import shutil
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from models import *
from tqdm import tqdm
from models.slimmableops import bn_calibration_init
import USconfig as FLAGS
import random

/home/hongky/.conda/envs/mdc/lib/python3.6/site-packages/setuptools/distutils_patch.py:26: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  "Distutils was imported before Setuptools. This usage is discouraged "


In [2]:
torch.cuda.is_available()

True

In [3]:
from dotmap import DotMap

args = DotMap()

args.dataset = 'imagenet'
args.batch_size=128
args.workers=4
args.test_batch_size=64
args.epochs = 1
args.start_epoch=0
args.lr = 0.2
args.momentum=0.9
args.weight_decay=1e-4
args.resume=''
args.no_cuda=False
args.seed=1
args.save='checkpoints_mobilenetv2_imagenet'
args.arch='MobileNetV2'
args.sr=True
args.s=0.0001
args.test=True


args.cuda = not args.no_cuda and torch.cuda.is_available()
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)


savepath = os.path.join(args.save, args.arch, 'sr' if args.sr else 'nosr')
if not os.path.exists(savepath):
    os.makedirs(savepath)
kwargs = {'num_workers': 4, 'pin_memory': True} if args.cuda else {}

args.data = '/home/hongky/datasets/imagenet'

In [4]:
# Data loading code
# traindir = os.path.join(args.data, 'train')
# valdir = os.path.join(args.data, 'val')
# normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                  std=[0.229, 0.224, 0.225])

# train_dataset = datasets.ImageFolder(
#     traindir,
#     transforms.Compose([
#         transforms.RandomResizedCrop(224),
#         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         normalize,
#     ]))

# train_loader = torch.utils.data.DataLoader(
#     train_dataset, batch_size=args.batch_size, shuffle=True,
#     num_workers=args.workers, pin_memory=True, sampler=None)

# test_loader = torch.utils.data.DataLoader(
#     datasets.ImageFolder(valdir, transforms.Compose([
#         transforms.Resize(256),
#         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#         normalize,
#     ])),
#     batch_size=args.batch_size, shuffle=False,
#     num_workers=args.workers, pin_memory=True)

In [ ]:
from torch.utils.data import DataLoader
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize,
])


test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize,
])




# Keep testset is the same for both teacher vs. student 
data_folder = args.data
train_folder = os.path.join(data_folder, 'train')
test_folder = os.path.join(data_folder, 'val')

if not os.path.exists(test_folder):
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), test_folder)

if not os.path.exists(train_folder):
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), train_folder)

# create dataset
train_set = datasets.ImageFolder(train_folder, transform=train_transform)
test_set = datasets.ImageFolder(test_folder, transform=test_transform)


#     shuffled_dataset = torch.utils.data.Subset(train_set, torch.randperm(len(train_set)).tolist())
num_workers = args.workers
batch_size = args.batch_size
train_loader = DataLoader(train_set,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=num_workers,
                          pin_memory=True,
                          sampler=None)

test_loader = DataLoader(test_set,
                         batch_size=batch_size,
                         shuffle=False,
                         num_workers=num_workers,
                         pin_memory=True)


In [ ]:
model = eval(args.arch)(n_class=1000, input_size=224)
print('Done eval model:', model)
if args.cuda:
    model.cuda()
best_prec1 = -1
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum, weight_decay=args.weight_decay)
if args.resume:
    if os.path.isfile(args.resume):
        print("=> loading checkpoint '{}'".format(args.resume))
        checkpoint = torch.load(args.resume)
        args.start_epoch = checkpoint['epoch']
        best_prec1 = checkpoint['best_prec1']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}' (epoch {}) Prec1: {:f}"
              .format(args.resume, checkpoint['epoch'], best_prec1))
    else:
        print("=> no checkpoint found at '{}'".format(args.resume))

In [ ]:
from torch.autograd import Variable

def updateBN():
    for m in model.modules():
        if isinstance(m, nn.BatchNorm2d):
            m.weight.grad.data.add_(args.s * torch.sign(m.weight.data))  # L1


def train():
    model.train()
    avg_loss = 0.
    train_acc = 0.
    
#     batch_time = AverageMeter()
#     losses = AverageMeter()
#     top1 = AverageMeter()
#     top5 = AverageMeter()
    
    
    for batch_idx, (data, target) in tqdm(enumerate(train_loader), total=len(train_loader)):
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        
        data = Variable(data)
        target = Variable(target)
        optimizer.zero_grad()
        
        
        output = model(data)
        
        loss = nn.CrossEntropyLoss()(output, target)
        #F.cross_entropy(output, target)
        #avg_loss += loss.item()
        #pred = output.data.max(1, keepdim=True)[1]
        #train_acc += pred.eq(target.data.view_as(pred)).sum()
        loss.backward()
        if args.sr:
            updateBN()
        optimizer.step()
        
        
def trainUS():
    max_width = max(FLAGS.width_mult_list)
    min_width = min(FLAGS.width_mult_list)

    model.train()
    for batch_idx, (data, target) in tqdm(enumerate(train_loader),total=len(train_loader)):
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        ###
        widths_train = []
        for _ in range(getattr(FLAGS, 'num_sample_training', 2) - 2):
            widths_train.append(
                random.uniform(min_width, max_width))
        widths_train = [max_width, min_width] + widths_train
        # widths_train = [min_width]
        for width_mult in widths_train:
            # TODO :add inplace distillation
            model.apply(lambda m: setattr(
                m, 'width_mult',
                width_mult))
                # always track largest model and smallest model
            output = model(data)
            loss = F.cross_entropy(output, target)
            loss.backward()
        ###
        optimizer.step()

def test(epoch,test_width=1.0,recal=False):
    model.eval()
    test_loss = 0
    correct = 0
    model.apply(lambda m: setattr(m, 'width_mult',test_width))
    if recal:
        model.apply(bn_calibration_init)
        model.train()
        for idx,(data, target) in enumerate(tqdm(train_loader, total=len(train_loader))):
            if idx==FLAGS.recal_batch:
                break
            if args.cuda:
                data, target = data.cuda(), target.cuda()
            with torch.no_grad():
                output = model(data)
            del output
            
    model.eval()
    for data, target in tqdm(test_loader, total=len(test_loader)):
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        with torch.no_grad():
            output = model(data)
        test_loss += F.cross_entropy(output, target, size_average=False).item()  # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1]  # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nEpoch: {} Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)\n'.format(epoch,
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return correct.item() / float(len(test_loader.dataset))

def export2normal():
    newmodel=MobileNetV2()
    from collections import OrderedDict
    statedic=[]
    for k2,v in model.state_dict().items():
        if 'running' in k2 or 'num_batches_tracked' in k2:
            continue
        statedic.append(v)
    names=[]
    for k1,v1 in newmodel.state_dict().items():
        if 'running' in k1 or 'num_batches_tracked' in k1:
            continue
        names.append(k1)
    newdic=OrderedDict(zip(names,statedic))
    newmodel.load_state_dict(newdic,strict=False)
    torch.save(newmodel.state_dict(),os.path.join(savepath,'trans.pth'))
    print("save transferred ckpt at {}".format(os.path.join(savepath,'trans.pth')))

In [ ]:
best_prec1 = 0. if best_prec1 == -1 else best_prec1
scheduler=optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max=args.epochs,eta_min=0)

for epoch in range(args.start_epoch, args.epochs):
#     if args.arch=='USMobileNetV2':
#         trainUS()
#         prec1=test(test_width=1.0,recal=False,epoch=epoch)
#     else:
    train()
    # prec1 = test(epoch=epoch)
    scheduler.step(epoch)
    lr_current = optimizer.param_groups[0]['lr']
#     print("currnt lr:{}".format(lr_current))
#     is_best = prec1 > best_prec1
#     best_prec1 = max(prec1, best_prec1)
#     if is_best:
#         ckptfile = os.path.join(savepath, 'model_best.pth.tar')
#     else:
    ckptfile = os.path.join(savepath, 'checkpoint.pth.tar')
        
    torch.save({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec1': best_prec1,
        'optimizer': optimizer.state_dict(),
    }, ckptfile)

In [ ]:
if args.arch=='USMobileNetV2':
    export2normal()
    res_acc=[1.0]*len(FLAGS.width_mult_list)
    for idx,width in enumerate(FLAGS.width_mult_list):
        acc=test(width,recal=True)
        res_acc[idx]=acc
        print("Test accuracy for width {} is {}".format(width,acc))
else:
    print("Test accuracy {}".format(test(0)))


In [ ]:
print(ckptfile)
torch.save({
    'epoch': epoch + 1,
    'state_dict': model.state_dict(),
    'best_prec1': best_prec1,
    'optimizer': optimizer.state_dict(),
}, ckptfile)